# Data cleaning
We want the class with labels as numerical value and the body with clean text.

This will remove:
* duplicates
* NaN entires
* non english
* url, html

* make it lowercase
* combine title and body

In [1]:
import pandas as pd
import sys
sys.path.append("../../../../../scripts_shared/")
from preprocess_text import preprocess_text


In [2]:
# Read CSV into a dataframe
df = pd.read_csv("../Jira.csv")
df

,id,fields.project.name,fields.priority.name,fields.created,fields.labels,fields.summary,fields.description,fields.status.name,fields.status.description,fields.issuetype.name,fields.issuetype.description,fields.issuetype.subtask,fields.comments
0,1843729,Sourcetree for Windows,NaN,2022-01-04T09:13:21.000+0000,[],Please make warning LESS SCARY about a detache...,I wanted to follow a perfectly normal and feas...,Gathering Interest,This suggestion needs more unique domain votes...,Suggestion,NaN,False,[]
1,1843575,Sourcetree for Windows,Low,2022-01-04T07:39:09.000+0000,[],Bug in listing bitbucket repos,some errors show up as shown in the screenshot...,Waiting for Release,A fix for this issue has been implemented and ...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...
2,1843552,Sourcetree for Windows,Low,2022-01-03T21:46:14.000+0000,[],Spaces in File path of Custom Action,I have been using Sourcetree 3.4.4. We use cu...,Needs Triage,This issue is waiting to be reviewed by a memb...,Bug,A problem which impairs or prevents the functi...,False,[]
3,1843397,Sourcetree for Windows,Low,2022-01-03T14:07:33.000+0000,[],Failed to start (System.BadImageFormatException),After installing SourceTree for Windows 10 64b...,Needs Triage,This issue is waiting to be reviewed by a memb...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...
4,1842733,Sourcetree for Windows,NaN,2021-12-25T02:30:54.000+0000,[],commit view show in branchs,When the project contains many branches and I ...,Gathering Interest,This suggestion needs more unique domain votes...,Suggestion,NaN,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
274540,1777577,Jira Service Management Cloud,NaN,2021-07-28T08:49:54.000+0000,[],Printing label tags for assets,Cant figure out a way to print labels on Insig...,Gathering Interest,This suggestion needs more unique domain votes...,Suggestion,NaN,False,[{'self': 'https://jira.atlassian.com/rest/api...
274541,1777505,Jira Service Management Cloud,Low,2021-07-28T07:18:51.000+0000,[],Having child objects with no individual attrib...,h3. Issue Summary\r\n\r\nCreating child object...,Long Term Backlog,"A fix for this issue is required, but planned ...",Bug,A problem which impairs or prevents the functi...,False,[]
274542,1777387,Jira Service Management Cloud,NaN,2021-07-28T06:50:23.000+0000,[],Ability to display Insight custom field in Kan...,h3. Problem\r\n\r\nCurrently there is no capab...,Gathering Interest,This suggestion needs more unique domain votes...,Suggestion,NaN,False,[]
274543,1777462,Jira Service Management Cloud,Low,2021-07-27T18:57:59.000+0000,[],Insight IQL uses timezones in a way that can b...,h3. Issue Summary\r\nInsight IQL uses timezone...,Gathering Impact,"This issue has been reviewed, but needs more s...",Bug,A problem which impairs or prevents the functi...,False,[]


In [3]:
# Count per priority
df['fields.priority.name'].value_counts()

fields.priority.name
Low        76547
Medium     43672
High       10283
Highest     5022
Name: count, dtype: int64

In [4]:
df['fields.issuetype.name'].value_counts().to_frame()[:50]

,count
fields.issuetype.name,
Suggestion,138378
Bug,131040
Sub-task,2501
Support Request,2438
Public Security Vulnerability,98
Improvement,52
New Feature,23
Task,12
Fug,2


In [5]:

highest = df[df['fields.priority.name'] == 'Highest']
high = df[df['fields.priority.name'] == 'High']
medium = df[df['fields.priority.name'] == 'Medium']
low = df[df['fields.priority.name'] == 'Low']
highest

,id,fields.project.name,fields.priority.name,fields.created,fields.labels,fields.summary,fields.description,fields.status.name,fields.status.description,fields.issuetype.name,fields.issuetype.description,fields.issuetype.subtask,fields.comments
551,590001,Sourcetree for Windows,Highest,2016-04-12T08:26:09.000+0000,[],Cannot complete the installation but get stuck...,Users cannot complete installing SourceTree bu...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[]
566,589080,Sourcetree for Windows,Highest,2016-04-08T08:53:27.000+0000,[],Crash on startup every time,Have been using sourcetree fine for some time ...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...
590,587950,Sourcetree for Windows,Highest,2016-04-05T13:38:49.000+0000,"['crash', 'submodule']",crash when git submodule has changes,In my git project I have changes in the main p...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...
609,587252,Sourcetree for Windows,Highest,2016-04-03T13:10:22.000+0000,"['clone', 'crash']",SourceTree consistently crashes when opening f...,When performing any action that should result ...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...
612,586936,Sourcetree for Windows,Highest,2016-04-01T11:05:45.000+0000,[],"""Create patch"" does not work properly on ""Work...","Whatever I do or select on ""Working Copy Chang...",Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
273661,322571,Jira Service Management Server and Data Center,Highest,2014-03-14T10:58:53.000+0000,[],Jira Service Desk. Error with license update,{panel:bgColor=#e7f4fa}\n *NOTE:* This bug re...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...
273813,313406,Jira Service Management Server and Data Center,Highest,2014-01-14T14:39:39.000+0000,['affects-server'],[java.lang.NullPointerException] Unable to run...,"Hello,\r\nwe have just test JIRA Service Desk,...",Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...
273887,307470,Jira Service Management Server and Data Center,Highest,2013-11-20T16:13:59.000+0000,[],Request types are created twice,{panel:bgColor=#e7f4fa}\n *NOTE:* This bug re...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...
273986,301979,Jira Service Management Server and Data Center,Highest,2013-10-13T22:27:20.000+0000,['affects-server'],Service Desk Email Notification From Address,Is the service desk using the notification ema...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...


In [6]:
# Concat high, medium and low
rest = pd.concat([medium, low])
rest

,id,fields.project.name,fields.priority.name,fields.created,fields.labels,fields.summary,fields.description,fields.status.name,fields.status.description,fields.issuetype.name,fields.issuetype.description,fields.issuetype.subtask,fields.comments
72,1788469,Sourcetree for Windows,Medium,2021-08-09T07:32:15.000+0000,"['notifications', 'verified', 'window']",Dictionary file blocks Sourcetree 2.3.1 from s...,"once when I was using Sourcetree v2.1.11, ever...",Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...
300,1185767,Sourcetree for Windows,Medium,2019-06-06T10:50:52.000+0000,['verified'],Drag/drop of local repositories does not work ...,I've organized my local repositories into fold...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...
504,592921,Sourcetree for Windows,Medium,2016-04-21T08:47:18.000+0000,"['OpenTerminal', 'crash']",Crash when opening Terminal,I was on a repository from https://github.com/...,Gathering Impact,"This issue has been reviewed, but needs more s...",Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...
510,592603,Sourcetree for Windows,Medium,2016-04-20T13:04:54.000+0000,[],Using Stash with Keep Staged Changes will stil...,If I have staged and unstaged changes and I cl...,Needs Triage,This issue is waiting to be reviewed by a memb...,Bug,A problem which impairs or prevents the functi...,False,[]
513,592492,Sourcetree for Windows,Medium,2016-04-20T06:36:56.000+0000,"['FileSystemWatcher', 'diff', 'refresh']",Impossible to inspect changesets in SourceTree,"In the history browser, select a changeset wit...",Needs Triage,This issue is waiting to be reviewed by a memb...,Bug,A problem which impairs or prevents the functi...,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
274537,1777664,Jira Service Management Cloud,Low,2021-07-29T01:42:19.000+0000,[],"Insight ""ORDER BY Updated"" IQL does not correc...",h3. Issue Summary\r\n\r\nSorting objects using...,Long Term Backlog,"A fix for this issue is required, but planned ...",Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...
274538,1777729,Jira Service Management Cloud,Low,2021-07-28T18:57:56.000+0000,[],customers page shows only 20 matched result fo...,h3. Issue Summary\r\n\r\nIf a service desk pro...,Long Term Backlog,"A fix for this issue is required, but planned ...",Bug,A problem which impairs or prevents the functi...,False,[]
274539,1777711,Jira Service Management Cloud,Low,2021-07-28T12:49:57.000+0000,[],Clicking on the Link issue button doesn't scro...,h3. Issue Summary\r\n\r\nWe have '*Link Issue*...,Gathering Impact,"This issue has been reviewed, but needs more s...",Bug,A problem which impairs or prevents the functi...,False,[]
274541,1777505,Jira Service Management Cloud,Low,2021-07-28T07:18:51.000+0000,[],Having child objects with no individual attrib...,h3. Issue Summary\r\n\r\nCreating child object...,Long Term Backlog,"A fix for this issue is required, but planned ...",Bug,A problem which impairs or prevents the functi...,False,[]


In [7]:
highest = pd.concat([highest, high])

In [8]:
#Give each priority a label by number.
# 'Label encoding'. Makes is easier for machine learning models to work with categorical data.

highest["label"] = 1
highest["class"] = "high_priority"
highest

,id,fields.project.name,fields.priority.name,fields.created,fields.labels,fields.summary,fields.description,fields.status.name,fields.status.description,fields.issuetype.name,fields.issuetype.description,fields.issuetype.subtask,fields.comments,label,class
551,590001,Sourcetree for Windows,Highest,2016-04-12T08:26:09.000+0000,[],Cannot complete the installation but get stuck...,Users cannot complete installing SourceTree bu...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[],1,high_priority
566,589080,Sourcetree for Windows,Highest,2016-04-08T08:53:27.000+0000,[],Crash on startup every time,Have been using sourcetree fine for some time ...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,1,high_priority
590,587950,Sourcetree for Windows,Highest,2016-04-05T13:38:49.000+0000,"['crash', 'submodule']",crash when git submodule has changes,In my git project I have changes in the main p...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,1,high_priority
609,587252,Sourcetree for Windows,Highest,2016-04-03T13:10:22.000+0000,"['clone', 'crash']",SourceTree consistently crashes when opening f...,When performing any action that should result ...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,1,high_priority
612,586936,Sourcetree for Windows,Highest,2016-04-01T11:05:45.000+0000,[],"""Create patch"" does not work properly on ""Work...","Whatever I do or select on ""Working Copy Chang...",Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[],1,high_priority
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273979,302243,Jira Service Management Server and Data Center,High,2013-10-15T10:40:08.000+0000,[],'Edit Form' doesn't work when using Russian la...,{panel:bgColor=#e7f4fa}\n *NOTE:* This bug re...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,1,high_priority
273981,302199,Jira Service Management Server and Data Center,High,2013-10-15T05:36:31.000+0000,['affects-server'],Restoring data in Oracle containing Service De...,The root cause of this issue is: https://ecosy...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[],1,high_priority
274153,1836986,Jira Service Management Cloud,High,2021-12-02T16:16:30.000+0000,[],Custom date range on reports shows a day prior...,h3. Issue Summary\r\n\r\nOn Service Management...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[],1,high_priority
274397,1804645,Jira Service Management Cloud,High,2021-09-15T06:05:43.000+0000,[],Workload Report in Service Project timing out,h3. Issue Summary:\r\n\r\nWorkload Report in S...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[],1,high_priority


In [9]:
rest["label"] = 0
rest["class"] = "not_high_priority"
rest

,id,fields.project.name,fields.priority.name,fields.created,fields.labels,fields.summary,fields.description,fields.status.name,fields.status.description,fields.issuetype.name,fields.issuetype.description,fields.issuetype.subtask,fields.comments,label,class
72,1788469,Sourcetree for Windows,Medium,2021-08-09T07:32:15.000+0000,"['notifications', 'verified', 'window']",Dictionary file blocks Sourcetree 2.3.1 from s...,"once when I was using Sourcetree v2.1.11, ever...",Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority
300,1185767,Sourcetree for Windows,Medium,2019-06-06T10:50:52.000+0000,['verified'],Drag/drop of local repositories does not work ...,I've organized my local repositories into fold...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority
504,592921,Sourcetree for Windows,Medium,2016-04-21T08:47:18.000+0000,"['OpenTerminal', 'crash']",Crash when opening Terminal,I was on a repository from https://github.com/...,Gathering Impact,"This issue has been reviewed, but needs more s...",Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority
510,592603,Sourcetree for Windows,Medium,2016-04-20T13:04:54.000+0000,[],Using Stash with Keep Staged Changes will stil...,If I have staged and unstaged changes and I cl...,Needs Triage,This issue is waiting to be reviewed by a memb...,Bug,A problem which impairs or prevents the functi...,False,[],0,not_high_priority
513,592492,Sourcetree for Windows,Medium,2016-04-20T06:36:56.000+0000,"['FileSystemWatcher', 'diff', 'refresh']",Impossible to inspect changesets in SourceTree,"In the history browser, select a changeset wit...",Needs Triage,This issue is waiting to be reviewed by a memb...,Bug,A problem which impairs or prevents the functi...,False,[],0,not_high_priority
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274537,1777664,Jira Service Management Cloud,Low,2021-07-29T01:42:19.000+0000,[],"Insight ""ORDER BY Updated"" IQL does not correc...",h3. Issue Summary\r\n\r\nSorting objects using...,Long Term Backlog,"A fix for this issue is required, but planned ...",Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority
274538,1777729,Jira Service Management Cloud,Low,2021-07-28T18:57:56.000+0000,[],customers page shows only 20 matched result fo...,h3. Issue Summary\r\n\r\nIf a service desk pro...,Long Term Backlog,"A fix for this issue is required, but planned ...",Bug,A problem which impairs or prevents the functi...,False,[],0,not_high_priority
274539,1777711,Jira Service Management Cloud,Low,2021-07-28T12:49:57.000+0000,[],Clicking on the Link issue button doesn't scro...,h3. Issue Summary\r\n\r\nWe have '*Link Issue*...,Gathering Impact,"This issue has been reviewed, but needs more s...",Bug,A problem which impairs or prevents the functi...,False,[],0,not_high_priority
274541,1777505,Jira Service Management Cloud,Low,2021-07-28T07:18:51.000+0000,[],Having child objects with no individual attrib...,h3. Issue Summary\r\n\r\nCreating child object...,Long Term Backlog,"A fix for this issue is required, but planned ...",Bug,A problem which impairs or prevents the functi...,False,[],0,not_high_priority


In [10]:
high_priority = highest
not_high_priority = rest

In [11]:
# Drop duplicates by the content of the description
high_priority = high_priority.drop_duplicates(subset=['fields.description'], keep='last')
high_priority.dropna(inplace=True)
high_priority.reset_index(inplace=True)
high_priority.drop(columns=["index"] , inplace= True)
high_priority["class"].value_counts()

/tmp/ipykernel_583685/1329060103.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_priority.dropna(inplace=True)
/tmp/ipykernel_583685/1329060103.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_priority.drop(columns=["index"] , inplace= True)


class
high_priority    14624
Name: count, dtype: int64

In [12]:
# Drop duplicates by the content of the title
not_high_priority = not_high_priority.drop_duplicates(subset=['fields.description'], keep='last')
not_high_priority.dropna(inplace=True)
not_high_priority.reset_index(inplace=True)
not_high_priority.drop(columns=["index"] , inplace= True)
not_high_priority["class"].value_counts()

/tmp/ipykernel_583685/2023168667.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_high_priority.dropna(inplace=True)
/tmp/ipykernel_583685/2023168667.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_high_priority.drop(columns=["index"] , inplace= True)


class
not_high_priority    114473
Name: count, dtype: int64

In [13]:
high_priority.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14624 entries, 0 to 14623
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   id                            14624 non-null  int64 
 1   fields.project.name           14624 non-null  object
 2   fields.priority.name          14624 non-null  object
 3   fields.created                14624 non-null  object
 4   fields.labels                 14624 non-null  object
 5   fields.summary                14624 non-null  object
 6   fields.description            14624 non-null  object
 7   fields.status.name            14624 non-null  object
 8   fields.status.description     14624 non-null  object
 9   fields.issuetype.name         14624 non-null  object
 10  fields.issuetype.description  14624 non-null  object
 11  fields.issuetype.subtask      14624 non-null  bool  
 12  fields.comments               14624 non-null  object
 13  label           

In [14]:
label_counts = high_priority["class"].value_counts()
label_counts_nhp = not_high_priority["class"].value_counts()
print(label_counts)
not_high_priority_count = label_counts_nhp["not_high_priority"]
print(not_high_priority_count)
hp_count = label_counts["high_priority"]
hp_count

class
high_priority    14624
Name: count, dtype: int64
114473


14624

In [15]:
not_high_priority = not_high_priority.sample(frac=hp_count/not_high_priority_count, random_state=42)
not_high_priority

,id,fields.project.name,fields.priority.name,fields.created,fields.labels,fields.summary,fields.description,fields.status.name,fields.status.description,fields.issuetype.name,fields.issuetype.description,fields.issuetype.subtask,fields.comments,label,class
108506,95742,Bamboo,Low,2009-10-16T00:01:48.000+0000,[],Bamboo deadlock on RemoteElasticInstance,Bamboo is dead. Long live Bamboo.,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[],0,not_high_priority
14831,580889,Confluence Server and Data Center,Medium,2016-03-11T20:30:04.000+0000,"['affects-cloud', 'affects-server', 'tcr']",When I want to delete a recurrency event I rec...,{panel:bgColor=#e7f4fa}\n *NOTE:* This bug re...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority
4750,82481,Jira Server and Data Center,Medium,2009-03-25T01:57:26.000+0000,"['affects-server', 'hedge']",Can't add a project to a customfield context w...,"In customfields, if you go to the configure sc...",Needs Triage,This issue is waiting to be reviewed by a memb...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority
56542,286339,Sourcetree for Windows,Low,2013-07-13T19:43:08.000+0000,[],"""Stage file"" resets the selected file to top o...","During the commit process, if you select ""Stag...",Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority
87111,104110,Confluence Server and Data Center,Low,2010-03-29T22:17:00.000+0000,"['3_3_bugfix_candidate', 'affects-server', 'bf...",Prevent NPE being thrown on recently updated d...,"When testing an upgrade, customer was getting ...",Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45872,830249,Jira Server and Data Center,Low,2017-08-10T13:07:35.000+0000,['rich-text-editor'],Changing issue types in issue creation screen ...,h3. Summary\r\n\r\nWhen changing the issue typ...,Gathering Impact,"This issue has been reviewed, but needs more s...",Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority
618,1423480,Jira Server and Data Center,Medium,2019-10-15T15:17:21.000+0000,"['mobile', 'warranty']",Jira mobile application is ignoring the contex...,h3. Issue Summary\r\n\r\nWhen context path is ...,Gathering Impact,"This issue has been reviewed, but needs more s...",Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority
79871,578716,Confluence Server and Data Center,Low,2016-03-04T14:37:26.000+0000,"['affects-server', 'editor', 'lists', 'pse-req...",Additional span section are added when copying...,h3. Summary\r\nConfluence editor adds multiple...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority
83753,276931,Confluence Server and Data Center,Low,2013-05-20T03:14:21.000+0000,"['affects-server', 'usability']",Identical Headers will not work when clicked i...,If a confluence page has two headers with the ...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority


In [16]:
all_priority = pd.concat([high_priority,not_high_priority] , ignore_index = True)
all_priority.tail()

,id,fields.project.name,fields.priority.name,fields.created,fields.labels,fields.summary,fields.description,fields.status.name,fields.status.description,fields.issuetype.name,fields.issuetype.description,fields.issuetype.subtask,fields.comments,label,class
29243,830249,Jira Server and Data Center,Low,2017-08-10T13:07:35.000+0000,['rich-text-editor'],Changing issue types in issue creation screen ...,h3. Summary\r\n\r\nWhen changing the issue typ...,Gathering Impact,"This issue has been reviewed, but needs more s...",Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority
29244,1423480,Jira Server and Data Center,Medium,2019-10-15T15:17:21.000+0000,"['mobile', 'warranty']",Jira mobile application is ignoring the contex...,h3. Issue Summary\r\n\r\nWhen context path is ...,Gathering Impact,"This issue has been reviewed, but needs more s...",Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority
29245,578716,Confluence Server and Data Center,Low,2016-03-04T14:37:26.000+0000,"['affects-server', 'editor', 'lists', 'pse-req...",Additional span section are added when copying...,h3. Summary\r\nConfluence editor adds multiple...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority
29246,276931,Confluence Server and Data Center,Low,2013-05-20T03:14:21.000+0000,"['affects-server', 'usability']",Identical Headers will not work when clicked i...,If a confluence page has two headers with the ...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority
29247,249943,Jira Cloud,Low,2012-12-28T11:06:09.000+0000,"['affects-cloud', 'jracloud-stale-triage']",layout in default project view,In attachment there's default project view lay...,Closed,Work on this issue is complete.\r\n\r\nIf it’s...,Bug,A problem which impairs or prevents the functi...,False,[{'self': 'https://jira.atlassian.com/rest/api...,0,not_high_priority


In [17]:
# 
all_priority["label"].value_counts()

label
1    14624
0    14624
Name: count, dtype: int64

In [18]:
print(all_priority["fields.description"][0])

Have been using sourcetree fine for some time and now it simply won't startup. Here's the call stack in the event log:

Application: SourceTree.exe
Framework Version: v4.0.30319
Description: The process was terminated due to an unhandled exception.
Exception Info: System.UnauthorizedAccessException
   at Microsoft.Win32.RegistryKey.Win32Error(Int32, System.String)
   at Microsoft.Win32.RegistryKey.SetValue(System.String, System.Object, Microsoft.Win32.RegistryValueKind)
   at SourceTree.AppRoot.RegisterBrowserEmulationModeKey(System.String, System.String, UInt32)
   at SourceTree.AppRoot.OnStartup(System.Windows.StartupEventArgs)
   at SourceTree.App.OnStartup(System.Windows.StartupEventArgs)
   at System.Windows.Application.<.ctor>b__1_0(System.Object)
   at System.Windows.Threading.ExceptionWrapper.InternalRealCall(System.Delegate, System.Object, Int32)
   at System.Windows.Threading.ExceptionWrapper.TryCatchWhen(System.Object, System.Delegate, System.Object, Int32, System.Delegate)


In [19]:
print(all_priority["fields.summary"][0])

Crash on startup every time


In [20]:
print(all_priority["fields.comments"][1])



[{'self': 'https://jira.atlassian.com/rest/api/2/issue/587950/comment/893927', 'id': '893927', 'author': {'self': 'https://jira.atlassian.com/rest/api/2/user?username=9168a1b5b7b7', 'name': '9168a1b5b7b7', 'key': 'lmtanco512499655', 'avatarUrls': {'48x48': 'https://jira.atlassian.com/secure/useravatar?ownerId=lmtanco512499655&avatarId=5513498', '24x24': 'https://jira.atlassian.com/secure/useravatar?size=small&ownerId=lmtanco512499655&avatarId=5513498', '16x16': 'https://jira.atlassian.com/secure/useravatar?size=xsmall&ownerId=lmtanco512499655&avatarId=5513498', '32x32': 'https://jira.atlassian.com/secure/useravatar?size=medium&ownerId=lmtanco512499655&avatarId=5513498'}, 'displayName': 'lmtanco', 'active': True, 'timeZone': 'Europe/Madrid'}, 'body': 'Same here :-)', 'updateAuthor': {'self': 'https://jira.atlassian.com/rest/api/2/user?username=9168a1b5b7b7', 'name': '9168a1b5b7b7', 'key': 'lmtanco512499655', 'avatarUrls': {'48x48': 'https://jira.atlassian.com/secure/useravatar?ownerId=l

In [21]:
# Make a new dataframe with only text, label and class cols.
all_priority_subset = all_priority[["fields.description" , "label" , "class"]]
all_priority_subset

,fields.description,label,class
0,Have been using sourcetree fine for some time ...,1,high_priority
1,In my git project I have changes in the main p...,1,high_priority
2,When performing any action that should result ...,1,high_priority
3,"Whatever I do or select on ""Working Copy Chang...",1,high_priority
4,Everything seems horribly slow including:\r\n*...,1,high_priority
...,...,...,...
29243,h3. Summary\r\n\r\nWhen changing the issue typ...,0,not_high_priority
29244,h3. Issue Summary\r\n\r\nWhen context path is ...,0,not_high_priority
29245,h3. Summary\r\nConfluence editor adds multiple...,0,not_high_priority
29246,If a confluence page has two headers with the ...,0,not_high_priority


In [22]:
# Convert to string
all_priority_subset["text_str"] = all_priority_subset['fields.description'].astype(str)

/tmp/ipykernel_583685/2820995913.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_str"] = all_priority_subset['fields.description'].astype(str)


In [23]:
all_priority_subset

,fields.description,label,class,text_str
0,Have been using sourcetree fine for some time ...,1,high_priority,Have been using sourcetree fine for some time ...
1,In my git project I have changes in the main p...,1,high_priority,In my git project I have changes in the main p...
2,When performing any action that should result ...,1,high_priority,When performing any action that should result ...
3,"Whatever I do or select on ""Working Copy Chang...",1,high_priority,"Whatever I do or select on ""Working Copy Chang..."
4,Everything seems horribly slow including:\r\n*...,1,high_priority,Everything seems horribly slow including:\r\n*...
...,...,...,...,...
29243,h3. Summary\r\n\r\nWhen changing the issue typ...,0,not_high_priority,h3. Summary\r\n\r\nWhen changing the issue typ...
29244,h3. Issue Summary\r\n\r\nWhen context path is ...,0,not_high_priority,h3. Issue Summary\r\n\r\nWhen context path is ...
29245,h3. Summary\r\nConfluence editor adds multiple...,0,not_high_priority,h3. Summary\r\nConfluence editor adds multiple...
29246,If a confluence page has two headers with the ...,0,not_high_priority,If a confluence page has two headers with the ...


In [24]:
# Clean the data.
all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)

/tmp/ipykernel_583685/63294665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)


In [25]:
# Make a subset with text_clean and label
priority_label_text = all_priority_subset[["text_clean" , "label"]]
priority_label_text

,text_clean,label
0,have been using sourcetree fine for some time ...,1
1,in my git project i have changes in the main p...,1
2,when performing any action that should result ...,1
3,whatever i do or select on working copy change...,1
4,everything seems horribly slow including many ...,1
...,...,...
29243,summary when changing the issue type in the cr...,0
29244,issue summary when context path is configured ...,0
29245,summary confluence editor adds multiple tags i...,0
29246,if a confluence page has two headers with the ...,0


In [26]:
# Row with NaN
priority_label_text[priority_label_text.isna().any(axis=1)]

,text_clean,label
8,NaN,1
153,NaN,1
203,NaN,1
354,NaN,1
359,NaN,1
...,...,...
28577,NaN,0
28918,NaN,0
29046,NaN,0
29094,NaN,0


In [27]:
# Need to dropna here since cleaning function returns NaN for not english text.
priority_label_text.dropna(inplace=True)
priority_label_text.reset_index(drop=True,inplace=True)

priority_label_text

/tmp/ipykernel_583685/2434149455.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.dropna(inplace=True)


,text_clean,label
0,have been using sourcetree fine for some time ...,1
1,in my git project i have changes in the main p...,1
2,when performing any action that should result ...,1
3,whatever i do or select on working copy change...,1
4,everything seems horribly slow including many ...,1
...,...,...
28871,summary when changing the issue type in the cr...,0
28872,issue summary when context path is configured ...,0
28873,summary confluence editor adds multiple tags i...,0
28874,if a confluence page has two headers with the ...,0


In [28]:
# Rows with NaN
priority_label_text[priority_label_text.isna().any(axis=1)]

,text_clean,label


In [29]:
from sklearn.utils import resample

# Split the dataset into two based on the label
df_majority = priority_label_text[priority_label_text['label'] == 0]
df_minority = priority_label_text[priority_label_text['label'] == 1]

# Undersample the majority class
df_majority_undersampled = resample(df_majority, 
                                     replace=False,    # sample without replacement
                                     n_samples=len(df_minority),     # to match minority class
                                     random_state=123) # reproducible results

# Combine minority class with downsampled majority class
df_balanced = pd.concat([df_majority_undersampled, df_minority])

# Shuffle the dataset to avoid any ordering bias
df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)
priority_label_text = df_balanced
priority_label_text["label"].value_counts()

label
1    14434
0    14434
Name: count, dtype: int64

In [30]:
priority_label_text[priority_label_text.isna().any(axis=1)]

,text_clean,label


In [31]:
null_rows = priority_label_text[priority_label_text['text_clean'].isnull()]
null_rows

,text_clean,label


In [32]:
# Clean dataset with clean text and labels.
# 0 = high priority, 1 = not high priority
file_name = f"clean_highest_and_high_vs_rest.csv"
priority_label_text.to_csv(file_name, index=False)

In [33]:
pri = pd.read_csv(file_name)
pri

,text_clean,label
0,i have the same problem as here but i update f...,1
1,the time estimate and time remaining values on...,0
2,note this bug report is for confluence server ...,1
3,steps to reproduce get a rapid view which is r...,0
4,i used the feed builder to create an rss feed ...,0
...,...,...
28863,issue summary connect app ui elements with the...,1
28864,issue summary group picker with a restricted c...,0
28865,navigation navigate to file status working cop...,1
28866,note this bug report is for confluence cloud u...,0


In [34]:
null_rows = pri[pri['text_clean'].isnull()]
null_rows

,text_clean,label
2796,NaN,0
2828,NaN,1
3359,NaN,0
9331,NaN,1
9740,NaN,1
15252,NaN,1
16523,NaN,1
19166,NaN,1
19331,NaN,0
20449,NaN,0


In [35]:
# Remove rows with NaN
pri = pri.dropna()
# Reset index
pri.reset_index(drop=True, inplace=True)
pri

,text_clean,label
0,i have the same problem as here but i update f...,1
1,the time estimate and time remaining values on...,0
2,note this bug report is for confluence server ...,1
3,steps to reproduce get a rapid view which is r...,0
4,i used the feed builder to create an rss feed ...,0
...,...,...
28851,issue summary connect app ui elements with the...,1
28852,issue summary group picker with a restricted c...,0
28853,navigation navigate to file status working cop...,1
28854,note this bug report is for confluence cloud u...,0


In [36]:
# save to csv
pri.to_csv(file_name, index=False)